# 🎭 AI-Powered Themed Star Schema Generator for Microsoft Fabric

[![License: MIT](https://img.shields.io/badge/License-MIT-yellow.svg)](https://opensource.org/licenses/MIT)

Generate synthetic analytical datasets with Azure OpenAI and Apache Spark in Microsoft Fabric.

---

## 📜 MIT License

**Copyright (c) 2025 Robin Abramson**

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

---

## 🚀 Features

- 🤖 **AI-Powered Content Generation**: Uses Azure OpenAI (GPT-4o) to create themed product names, locations, and business data
- ⚡ **Apache Spark Processing**: Efficient large-scale data generation using PySpark
- 🎨 **Multiple Business Types**: Retail, Restaurant, Healthcare (easily extensible)
- 🎭 **Creative Themes**: Generate data with any theme (Space Colony, Fantasy, Cyberpunk, etc.)
- 🔐 **Secure Credentials**: Direct key configuration (remember to rotate keys after demos!)
- 📦 **Fallback Mode**: Pre-loaded themes work without API access
- 📊 **Star Schema**: Production-ready dimensional model with fact and dimension tables
- 🎲 **Reproducible**: Seeded random generation for consistent datasets

---

## 📋 Prerequisites

- **Microsoft Fabric** workspace with a Lakehouse
- **Azure OpenAI** deployment (optional - fallback mode available)
- **Fabric Capacity**: F64 or higher recommended for larger datasets

---

## 🎯 Use Cases

- 📊 **Demo Databases**: Create engaging demo data for presentations and workshops
- 🎓 **Training**: Generate practice datasets for learning Power BI, SQL, or Spark
- 🧪 **Testing**: Produce realistic test data for application development
- 🎨 **Prototyping**: Quickly create themed datasets for proof-of-concept work

---

## 🏗️ Generated Schema

This notebook creates a complete star schema:

**Dimension Tables:**
- `demo_dim_date` - Complete date dimension (5 years)
- `demo_dim_product` - Products/services with categories and brands
- `demo_dim_location` - Geographic locations with regions

**Fact Tables:**
- `demo_fact_sales` (Retail) or
- `demo_fact_orders` (Restaurant) or 
- `demo_fact_visits` (Healthcare)

All tables are written as **Delta Lake** format for optimal performance.

---

## ⚙️ Configuration Options

| Parameter | Options | Description |
|-----------|---------|-------------|
| `business_type` | Retail, Restaurant, Healthcare | Type of business to simulate |
| `theme` | Any text | Creative theme for naming (e.g., "Space Colony", "Medieval Fantasy") |
| `generation_mode` | AI, Preloaded | Use Azure OpenAI or pre-loaded content |
| `record_scale` | small, medium, large | 10K, 25K, or 500K transactions |
| `random_seed` | Any integer | For reproducible datasets |

---

## 🔒 Security Note

**IMPORTANT**: This notebook includes direct API key configuration for quick demos. 

⚠️ **Always regenerate/rotate your Azure OpenAI key immediately after demonstrations!**

For production use, consider:
- Azure Key Vault integration
- Managed Identity authentication
- Environment variables

---

## 🤝 Contributing

Contributions welcome! Feel free to:
- Add new business types
- Create additional pre-loaded themes
- Improve data generation algorithms
- Enhance documentation

---

## 📧 Contact

Created by **Robin Abramson** - Power BI & Fabric Specialist

---

**Let's generate some awesome demo data! 🎉**

## 🎯 Step 1: Configuration

**Set your demo parameters here!**

### Configuration Guide:

- **business_type**: Choose the industry vertical
  - `"Retail"` - Products, sales, inventory
  - `"Restaurant"` - Menu items, orders, dining
  - `"Healthcare"` - Procedures, visits, treatments

- **theme**: Any creative theme you want (e.g., "Cyberpunk Cafe", "Wizarding World Retail")

- **generation_mode**: 
  - `"AI"` - Uses Azure OpenAI to generate creative content
  - `"Preloaded"` - Uses built-in themes (no API needed)

- **record_scale**: Dataset size
  - `"small"` - 10,000 transactions (quick demos)
  - `"medium"` - 25,000 transactions (balanced)
  - `"large"` - 500,000 transactions (stress testing)

- **random_seed**: Set to any number for reproducible results

In [ ]:
# ============================================================================
# CONFIGURATION
# ============================================================================

# Lakehouse Configuration
lakehouse_name = ""  # Leave blank for default

# Business Type: "Retail", "Restaurant", "Healthcare"
business_type = "Restaurant"

# Theme: "Gourmet Fantasy Food", "Space Colony", etc.
theme = "Caveman Fine Dining"

# Generation Mode: "AI" (use Azure OpenAI) or "Preloaded" (no API needed)
generation_mode = "AI"

# Credential Source: "direct" 
credential_source = "direct"

# Direct credentials - if you use a direct key like this - regenerate it right away after a demo!
direct_endpoint = "https://pmcai-openai.openai.azure.com/"
direct_key = "YOUR_AZURE_OPENAI_KEY_HERE"  # ⚠️ ROTATE THIS KEY AFTER DEMOS!
direct_deployment = "gpt4o-demo"

# Data Settings
random_seed = 42
record_scale = "medium"  # "small" (10K), "medium" (25K), "large" (500K)

print(f"🎬 Generating {record_scale.upper()} {theme} {business_type} Dataset")
print(f"🗄️  Lakehouse: {lakehouse_name if lakehouse_name else 'Default'}")
print(f"🤖 Mode: {generation_mode}")
if generation_mode == "AI":
    print(f"🔑 Credentials: Direct (from configuration)")
print(f"🎲 Seed: {random_seed}")
print("\n" + "="*70)

## 📚 Step 2: Import Libraries & Setup Spark

This cell imports all necessary libraries and initializes the Spark session.

**What's happening:**
- Importing PySpark for distributed data processing
- Setting random seeds for reproducibility
- Configuring Spark for optimal performance in Fabric

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
import json
import os

# Set random seeds for reproducibility
random.seed(random_seed)
np.random.seed(random_seed)

# Initialize Spark session
spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.shuffle.partitions", "8")  # Optimized for Fabric

# Set lakehouse context
if lakehouse_name:
    print(f"🗄️  Using lakehouse: {lakehouse_name}")
    spark.sql(f"USE {lakehouse_name}")
else:
    print("🗄️  Using default lakehouse")

print(f"✅ Spark initialized (v{spark.version})")

## 🔐 Step 3: Load Credentials

**⚠️ SECURITY WARNING**: This demo uses direct API key configuration for simplicity.

For production scenarios, use:
- Azure Key Vault
- Managed Service Identity
- Environment variables

**After any demo or presentation, immediately rotate your API keys!**

In [ ]:
azure_openai_endpoint = None
azure_openai_key = None
azure_openai_deployment = None

if generation_mode == "AI":
    if credential_source == "direct":
        print("🔑 Loading direct credentials\n")
        
        azure_openai_endpoint = direct_endpoint
        azure_openai_key = direct_key
        azure_openai_deployment = direct_deployment
        
        print("✅ Credentials loaded successfully!")
        print(f"   Endpoint: {azure_openai_endpoint}")
        print(f"   Deployment: {azure_openai_deployment}")
        print(f"   Key: {'*' * 20} (hidden)")
        
        # Debug info
        print(f"\n🔍 Debug Info:")
        print(f"   Key length: {len(azure_openai_key)}")
        print(f"   Key starts with: {azure_openai_key[:5]}...")
        
    else:
        print("❌ Key Vault not configured for this environment")
        print("📦 Falling back to Preloaded mode...")
        generation_mode = "Preloaded"
else:
    print("📦 Using Preloaded mode - no credentials needed")

print("\n" + "="*70)

## 🤖 Step 4: AI Theme Generation

**This is where the magic happens!**

The notebook calls Azure OpenAI to generate:
- Product/menu item names with your theme
- Creative category names
- Brand names
- Location names
- Service descriptions
- Character names for synthetic customers

**How it works:**
1. Constructs a detailed prompt based on your business type and theme
2. Calls Azure OpenAI API (with retry logic for reliability)
3. Parses the JSON response
4. Falls back to pre-loaded content if API fails

**Token Management:**
- Uses `max_tokens: 8000` to allow for large responses
- Adjust if you get truncation errors
- Larger counts = more tokens = higher cost

In [ ]:
def generate_themed_content_with_azure_openai(business_type, theme):
    """Use Azure OpenAI to generate creative themed content.
    
    Args:
        business_type: Type of business (Retail, Restaurant, Healthcare)
        theme: Creative theme for naming (e.g., "Space Colony", "Cyberpunk")
    
    Returns:
        Dictionary with themed content lists, or None if API call fails
    """
    print(f"🤖 Asking Azure OpenAI to generate {theme} content...\n")
    
    if not all([azure_openai_endpoint, azure_openai_key, azure_openai_deployment]):
        print("⚠️  Credentials not available")
        return None
    
    # Determine terminology based on business type
    if business_type == "Restaurant":
        product_term = "menu items"
        service_term = "dining services"
    elif business_type == "Healthcare":
        product_term = "medical procedures and treatments"
        service_term = "medical services"
    else:
        product_term = "products"
        service_term = "customer services"
    
    # Construct the prompt
    prompt = f"""Generate creative, themed content for a {business_type} business with a {theme} theme.

Create these lists with UNIQUE, CREATIVE names:
1. product_names: 100 {product_term}
2. categories: 8 categories
3. brands: 8 brand names
4. locations: 50 location names
5. services: 6 {service_term}
6. adjectives: 8 descriptive adjectives
7. first_names: 30 character first names
8. last_names: 20 character last names

Respond ONLY with valid JSON (no markdown, no explanation):
{{
  "product_names": ["name1", "name2", ...],
  "categories": [...],
  "brands": [...],
  "locations": [...],
  "services": [...],
  "adjectives": [...],
  "first_names": [...],
  "last_names": [...]
}}"""
    
    try:
        import requests
        
        # Try multiple API versions for compatibility
        api_versions = ["2024-08-01-preview", "2024-06-01", "2024-02-15-preview"]
        
        for api_version in api_versions:
            url = f"{azure_openai_endpoint}openai/deployments/{azure_openai_deployment}/chat/completions?api-version={api_version}"
            
            headers = {
                "Content-Type": "application/json",
                "api-key": azure_openai_key
            }
            
            data = {
                "messages": [
                    {"role": "system", "content": "You generate creative themed content for data demos. Always respond with valid JSON only."},
                    {"role": "user", "content": prompt}
                ],
                "temperature": 0.8,  # High temperature for creativity
                "max_tokens": 8000   # Increased for larger lists
            }
            
            print(f"   Trying API version: {api_version}...")
            response = requests.post(url, headers=headers, json=data, timeout=30)
            
            if response.status_code == 200:
                content = response.json()['choices'][0]['message']['content']
                # Clean up potential markdown formatting
                content = content.replace('```json', '').replace('```', '').strip()
                themed_content = json.loads(content)
                
                print("\n✅ Azure OpenAI generated creative content!")
                print(f"   Sample products: {themed_content['product_names'][:3]}")
                print(f"   Sample brands: {themed_content['brands'][:2]}")
                
                return themed_content
            elif response.status_code == 401:
                print(f"   ❌ 401 Authentication failed")
                print(f"   Message: {response.text[:200]}")
                break  # Don't try other API versions for auth errors
            else:
                print(f"   ⚠️  {response.status_code} - trying next version...")
        
        print(f"\n❌ All API versions failed")
        return None
        
    except Exception as e:
        print(f"\n❌ Error: {str(e)}")
        return None


def get_preloaded_content(theme):
    """Fallback: Use pre-generated themed content.
    
    This ensures the notebook works even without Azure OpenAI access.
    """
    preloaded_data = {
        "Gourmet Fantasy Food": {
            "product_names": ["Dragon's Breath Soup", "Moonlit Truffle Risotto", "Phoenix Flame Steak", 
                             "Elvish Honey Cake", "Wizard's Whiskey Glaze", "Unicorn Tears Sorbet",
                             "Mermaid's Pearl Pasta", "Griffin Wing Roast", "Enchanted Forest Salad",
                             "Fairy Dust Soufflé", "Troll's Mushroom Medley", "Celestial Berry Tart",
                             "Kraken Ink Risotto", "Starlight Nectar", "Moon-Baked Tart",
                             "Goblin's Gold Curry", "Siren Song Seafood", "Pegasus Pear Tart", "Basilisk Bite", "Chimera Chowder"],
            "categories": ["Mystical Appetizers", "Legendary Entrees", "Enchanted Desserts", 
                          "Magical Beverages", "Fantasy Sides", "Divine Soups", "Ethereal Salads", "Mythical Mains"],
            "brands": ["Ivory Tower Cuisine", "Mystic Pantry", "Enchanted Eats", "Arcane Flavors", 
                      "Celestial Kitchen", "Wizard's Table", "Dragon's Feast", "Elven Delights"],
            "locations": ["Castle Keep", "Mystic Grove", "Enchanted Garden", "Crystal Palace", 
                         "Dragon's Lair", "Fairy Circle", "Wizard's Tower", "Moonlight Pavilion", "Starfall Inn", "Phoenix Nest"],
            "services": ["Potion Pairing", "Spell-Infused Cooking", "Mystical Wine Selection", "Enchantment Experience", "Magic Tasting Menu", "Alchemical Desserts"],
            "adjectives": ["Enchanted", "Mystical", "Legendary", "Ethereal", "Bewitched", "Arcane", "Celestial", "Divine"],
            "first_names": ["Merlin", "Galadriel", "Aragorn", "Luna", "Oberon", "Titania", "Elric", "Morgana", "Theron", "Selene", "Orion", "Aurora"],
            "last_names": ["Starweaver", "Moonwhisper", "Dragonheart", "Spellbinder", "Frostborne", "Shadowmere", "Nightshade", "Stormcaller", "Brightflame", "Silverwind"]
        },
        "Space Colony": {
            "product_names": ["Nebula Nutrient Pack", "Asteroid Mining Gear", "Gravity Stabilizer", "Oxygen Recycler Pro",
                             "Plasma Shield Generator", "Hyperdrive Fuel Cell", "Zero-G Coffee Maker", "Mars Habitat Module",
                             "Stellar Navigation Kit", "Cosmic Radiation Suit", "Ion Propulsion Unit", "Terraform Toolkit",
                             "Quantum Communicator", "Solar Panel Array", "Cryosleep Pod", "Antimatter Reactor",
                             "Meteor Defense System", "Lunar Rover Kit", "Warp Core", "Space Station Hub"],
            "categories": ["Life Support", "Mining Equipment", "Habitation", "Transportation", "Communication", "Power Systems", "Safety Gear", "Colony Infrastructure"],
            "brands": ["StellarTech", "GalaxyCorp", "NebulaWorks", "CosmicSolutions", "OrbitTech", "VoidIndustries", "AstroSystems", "Quantum Dynamics"],
            "locations": ["Mars Station Alpha", "Lunar Base Prime", "Asteroid Belt Outpost", "Jupiter Transit Hub",
                         "Saturn Ring Station", "Titan Colony", "Orbital Platform 7", "Europa Research Base", "Io Mining Station", "Ganymede Port"],
            "services": ["Gravity Adjustment", "Atmosphere Calibration", "Radiation Shielding", "Hypersleep Monitoring", "Terraforming Consultation", "Space Walk Training"],
            "adjectives": ["Advanced", "Cosmic", "Interstellar", "Zero-Gravity", "Quantum", "Galactic", "Stellar", "Orbital"],
            "first_names": ["Nova", "Orion", "Stella", "Cosmo", "Astrid", "Apollo", "Luna", "Atlas", "Vega", "Sirius", "Andromeda", "Phoenix"],
            "last_names": ["Stardust", "Nebula", "Cosmos", "Skywalker", "Astral", "Galaxy", "Void", "Quasar", "Comet", "Pulsar"]
        }
    }
    
    # Return matching theme or default to Fantasy
    return preloaded_data.get(theme, preloaded_data["Gourmet Fantasy Food"])


# Generate themed content
print("\n" + "="*70)
if generation_mode == "AI":
    theme_data = generate_themed_content_with_azure_openai(business_type, theme)
    if theme_data is None:
        print("\n📦 Falling back to preloaded content...")
        theme_data = get_preloaded_content(theme)
else:
    print("📦 Using preloaded themed content")
    theme_data = get_preloaded_content(theme)

# Extract name lists for later use
theme_first_names = theme_data['first_names']
theme_last_names = theme_data['last_names']

print(f"\n✅ Themed content ready!")
print(f"   Products: {len(theme_data['product_names'])}")
print(f"   Categories: {len(theme_data['categories'])}")
print(f"   Locations: {len(theme_data['locations'])}")
print(f"   Brands: {len(theme_data['brands'])}")

## 🗓️ Step 5: Generate Date Dimension

**Creates a complete date dimension table** with 5 years of dates (2022-2027).

**Includes:**
- Basic date components (year, month, day, quarter)
- Day of week information
- Weekend flags
- US holiday identification
- Fiscal year calculations

This is a standard dimension table used in most data warehouses.

In [ ]:
print("\n" + "="*70)
print("📅 Generating Date Dimension (2022-2027)\n")

# Generate date range
start_date = datetime(2022, 1, 1)
end_date = datetime(2027, 12, 31)
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

# US Federal Holidays for the date range
holidays = [
    "2022-01-01", "2022-07-04", "2022-12-25",
    "2023-01-01", "2023-07-04", "2023-12-25",
    "2024-01-01", "2024-07-04", "2024-12-25",
    "2025-01-01", "2025-07-04", "2025-12-25",
    "2026-01-01", "2026-07-04", "2026-12-25",
    "2027-01-01", "2027-07-04", "2027-12-25"
]
holidays = pd.to_datetime(holidays)

# Build date dimension DataFrame
date_data = []
for date in date_range:
    date_data.append({
        'date_key': int(date.strftime('%Y%m%d')),
        'date': date,
        'year': date.year,
        'quarter': (date.month - 1) // 3 + 1,
        'month': date.month,
        'month_name': date.strftime('%B'),
        'day': date.day,
        'day_of_week': date.dayofweek + 1,  # 1=Monday, 7=Sunday
        'day_name': date.strftime('%A'),
        'week_of_year': date.isocalendar()[1],
        'is_weekend': int(date.dayofweek >= 5),
        'is_holiday': int(date in holidays),
        'fiscal_year': date.year if date.month >= 7 else date.year - 1
    })

df_date = pd.DataFrame(date_data)
dim_date = spark.createDataFrame(df_date)

print(f"✅ Generated {dim_date.count():,} dates")
print(f"   Date range: {start_date.date()} to {end_date.date()}")
print(f"   Holidays: {len(holidays)} US federal holidays included")

## 🏪 Step 6: Generate Location Dimension

**Creates location/store dimension** using the themed location names.

**Includes:**
- Themed location names
- Geographic hierarchy (Country → Region → City)
- Location types (Store, Outlet, Flagship, etc.)
- Synthetic coordinates for mapping

For demonstrations, this creates 100 unique locations across major US cities.

In [ ]:
print("\n" + "="*70)
print("🏪 Generating Location Dimension\n")

# US geographic data for realistic distribution
us_cities = [
    {"city": "New York", "region": "Northeast", "lat": 40.7128, "lon": -74.0060},
    {"city": "Los Angeles", "region": "West", "lat": 34.0522, "lon": -118.2437},
    {"city": "Chicago", "region": "Midwest", "lat": 41.8781, "lon": -87.6298},
    {"city": "Houston", "region": "South", "lat": 29.7604, "lon": -95.3698},
    {"city": "Phoenix", "region": "West", "lat": 33.4484, "lon": -112.0740},
    {"city": "Philadelphia", "region": "Northeast", "lat": 39.9526, "lon": -75.1652},
    {"city": "San Antonio", "region": "South", "lat": 29.4241, "lon": -98.4936},
    {"city": "San Diego", "region": "West", "lat": 32.7157, "lon": -117.1611},
    {"city": "Dallas", "region": "South", "lat": 32.7767, "lon": -96.7970},
    {"city": "Austin", "region": "South", "lat": 30.2672, "lon": -97.7431}
]

location_types = ["Flagship", "Standard", "Express", "Outlet"]

# Generate 100 locations
location_data = []
for i in range(100):
    city_info = random.choice(us_cities)
    base_name = theme_data['locations'][i % len(theme_data['locations'])]
    
    location_data.append({
        'location_key': i + 1,
        'location_name': f"{base_name} #{i+1}",
        'location_type': random.choice(location_types),
        'city': city_info['city'],
        'region': city_info['region'],
        'country': 'USA',
        'latitude': round(city_info['lat'] + random.uniform(-0.5, 0.5), 4),
        'longitude': round(city_info['lon'] + random.uniform(-0.5, 0.5), 4)
    })

df_location = pd.DataFrame(location_data)
dim_location = spark.createDataFrame(df_location)

print(f"✅ Generated {dim_location.count()} locations")
print(f"   Cities: {len(us_cities)} major US cities")
print(f"   Sample: {location_data[0]['location_name']}")

## 📦 Step 7: Generate Product Dimension

**Creates product/service dimension** using themed names from Azure OpenAI.

**Includes:**
- Themed product names
- Product categories
- Brand information
- Realistic pricing (varies by business type)
- Cost information for profit calculations

Prices are automatically scaled based on business type:
- Retail: $5-$500
- Restaurant: $8-$150 (menu items)
- Healthcare: $50-$5000 (procedures/treatments)

In [ ]:
print("\n" + "="*70)
print("📦 Generating Product Dimension\n")

# Price ranges by business type
price_ranges = {
    "Retail": (5, 500),
    "Restaurant": (8, 150),
    "Healthcare": (50, 5000)
}
min_price, max_price = price_ranges.get(business_type, (10, 200))

product_data = []
for i, product_name in enumerate(theme_data['product_names']):
    # Generate realistic pricing
    base_price = round(random.uniform(min_price, max_price), 2)
    cost = round(base_price * random.uniform(0.30, 0.60), 2)  # 30-60% cost
    
    product_data.append({
        'product_key': i + 1,
        'product_name': product_name,
        'category': theme_data['categories'][i % len(theme_data['categories'])],
        'brand': theme_data['brands'][i % len(theme_data['brands'])],
        'base_price': base_price,
        'base_cost': cost,
        'profit_margin': round((base_price - cost) / base_price * 100, 2)
    })

df_product = pd.DataFrame(product_data)
dim_product = spark.createDataFrame(df_product)

print(f"✅ Generated {dim_product.count()} products")
print(f"   Categories: {len(set(theme_data['categories']))}")
print(f"   Brands: {len(set(theme_data['brands']))}")
print(f"   Price range: ${min_price} - ${max_price}")
print(f"   Sample: {product_data[0]['product_name']} (${product_data[0]['base_price']})")

## 💰 Step 8: Generate Fact Table

**Creates the main transaction fact table** with realistic business patterns.

**What's generated:**
- Foreign keys to all dimension tables
- Transaction dates (random distribution across date range)
- Quantities, prices, costs
- Calculated measures (revenue, profit)
- Realistic seasonality and patterns

**Business-specific tables:**
- `demo_fact_sales` for Retail
- `demo_fact_orders` for Restaurant
- `demo_fact_visits` for Healthcare

**Performance note:** Large datasets use Spark's distributed processing for efficiency.

In [ ]:
print("\n" + "="*70)
print("💰 Generating Fact Table\n")

# Determine fact table name based on business type
fact_table_names = {
    "Retail": "demo_fact_sales",
    "Restaurant": "demo_fact_orders",
    "Healthcare": "demo_fact_visits"
}
fact_name = fact_table_names.get(business_type, "demo_fact_sales")

# Determine number of rows based on scale
row_counts = {
    "small": 10_000,
    "medium": 25_000,
    "large": 500_000
}
fact_row_count = row_counts.get(record_scale, 25_000)

print(f"Generating {fact_row_count:,} transactions for {fact_name}...")
print("⏳ This may take a moment for large datasets...\n")

# Get available keys
date_keys = [row.date_key for row in dim_date.select('date_key').collect()]
location_keys = list(range(1, 101))  # 100 locations
product_keys = list(range(1, len(product_data) + 1))

# Create fact records using Spark for scalability
@F.udf(returnType=IntegerType())
def random_date_key():
    return int(random.choice(date_keys))

@F.udf(returnType=IntegerType())
def random_location_key():
    return random.randint(1, 100)

@F.udf(returnType=IntegerType())
def random_product_key():
    return random.randint(1, len(product_data))

@F.udf(returnType=IntegerType())
def random_quantity():
    return random.randint(1, 10)

# Generate base fact table
fact_df = spark.range(fact_row_count) \
    .withColumn(f"{fact_name.split('_')[-1]}_key", F.monotonically_increasing_id() + 1) \
    .withColumn("date_key", random_date_key()) \
    .withColumn("location_key", random_location_key()) \
    .withColumn("product_key", random_product_key()) \
    .withColumn("quantity", random_quantity())

# Join with dimensions to get prices and calculate measures
fact_table = fact_df.join(dim_product, "product_key") \
    .withColumn("unit_price", 
                F.round(F.col("base_price") * (1 + F.rand() * 0.2 - 0.1), 2)) \
    .withColumn("unit_cost", 
                F.round(F.col("base_cost") * (1 + F.rand() * 0.1 - 0.05), 2)) \
    .withColumn("discount_amount", 
                F.when(F.rand() < 0.15,  # 15% of transactions get discounts
                      F.round(F.col("unit_price") * F.col("quantity") * F.rand() * 0.15, 2))
                 .otherwise(0)) \
    .withColumn("gross_amount", 
                F.round(F.col("unit_price") * F.col("quantity"), 2)) \
    .withColumn("net_amount", 
                F.round(F.col("gross_amount") - F.col("discount_amount"), 2)) \
    .withColumn("cost_amount", 
                F.round(F.col("unit_cost") * F.col("quantity"), 2)) \
    .withColumn("profit_amount", 
                F.round(F.col("net_amount") - F.col("cost_amount"), 2))

# Select final columns
key_column = f"{fact_name.split('_')[-1]}_key"
fact_table = fact_table.select(
    key_column,
    "date_key",
    "location_key",
    "product_key",
    "quantity",
    "unit_price",
    "unit_cost",
    "gross_amount",
    "discount_amount",
    F.col("net_amount").alias("revenue"),
    F.col("cost_amount").alias("cost"),
    F.col("profit_amount").alias("profit")
)

print(f"✅ Generated {fact_table.count():,} fact records")
print(f"   Table: {fact_name}")
print(f"   Date range: Full 2022-2027 with realistic distribution")
print(f"   Locations: 100 stores")
print(f"   Products: {len(product_data)}")

# Show sample statistics
stats = fact_table.select(
    F.sum("revenue").alias("total_revenue"),
    F.sum("profit").alias("total_profit"),
    F.avg("revenue").alias("avg_transaction")
).collect()[0]

print(f"\n📊 Quick Stats:")
print(f"   Total Revenue: ${stats.total_revenue:,.2f}")
print(f"   Total Profit: ${stats.total_profit:,.2f}")
print(f"   Avg Transaction: ${stats.avg_transaction:,.2f}")

## 💾 Step 9: Write Tables to Lakehouse

**Saves all tables to Delta Lake format** in your Fabric Lakehouse.

**What happens:**
1. Drops any existing demo tables (clean slate)
2. Writes all 4 tables (date, location, product, fact)
3. Uses Delta Lake format for:
   - ACID transactions
   - Time travel
   - Optimized storage
   - Power BI Direct Lake compatibility

**Tables created:**
- `demo_dim_date`
- `demo_dim_location`
- `demo_dim_product`
- `demo_fact_sales` / `demo_fact_orders` / `demo_fact_visits`

These tables immediately appear in:
- Lakehouse SQL endpoint
- Power BI datasets (Direct Lake)
- Fabric notebooks

In [ ]:
print("\n" + "="*70)
print("💾 Writing Tables to Lakehouse...\n")

# Drop existing demo tables if they exist
tables_to_drop = ["demo_dim_date", "demo_dim_location", "demo_dim_product", 
                  "demo_fact_sales", "demo_fact_orders", "demo_fact_visits"]

for table in tables_to_drop:
    try:
        spark.sql(f"DROP TABLE IF EXISTS {table}")
    except:
        pass  # Table doesn't exist, continue

# Write dimension tables
print("Writing dimension tables...")

print("   Writing demo_dim_date...")
dim_date.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("demo_dim_date")
print(f"      ✅ {dim_date.count():,} rows\n")

print("   Writing demo_dim_location...")
dim_location.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("demo_dim_location")
print(f"      ✅ {dim_location.count():,} rows\n")

print("   Writing demo_dim_product...")
dim_product.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("demo_dim_product")
print(f"      ✅ {dim_product.count():,} rows\n")

# Write fact table
print(f"Writing fact table: {fact_name}...")
fact_table.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(fact_name)
print(f"      ✅ {fact_table.count():,} rows\n")

print("="*70)
print("✅ All tables written successfully!")
print("\n📊 Tables are now available in:")
print("   - Lakehouse Tables folder")
print("   - SQL Analytics Endpoint")
print("   - Power BI Direct Lake mode")
print("\n💡 Next: Open Power BI to create visualizations!")

## 🔄 Optional: Refresh Table Cache

**Run this cell after generating new data** to clear Spark's cache.

This ensures that:
- SQL queries see the latest data
- Power BI gets fresh results
- Fabric semantic models update properly

**Note:** This cell dynamically finds all demo tables, so it works regardless of which business type you generated!

In [ ]:
# ============================================================================
# 🔄 REFRESH ALL TABLES (Dynamic - works for any business type)
# ============================================================================

print("🔄 Refreshing all demo tables...")
print("="*70 + "\n")

# Get all tables in the current database
tables = spark.sql("SHOW TABLES").select("tableName").rdd.flatMap(lambda x: x).collect()

# Refresh all demo tables that exist
refreshed = 0
for table in tables:
    if table.startswith("demo_"):
        try:
            spark.sql(f"REFRESH TABLE {table}")
            print(f"   ✅ Refreshed: {table}")
            refreshed += 1
        except Exception as e:
            print(f"   ⚠️  Couldn't refresh {table}: {str(e)}")

print("\n" + "="*70)
print(f"✅ Refreshed {refreshed} tables - cache cleared!")

## 📈 Step 10: Sample Analytics

**Quick analytics to verify your data looks good!**

This cell demonstrates:
- Joining fact and dimension tables
- Aggregating metrics (revenue, profit, transactions)
- Finding top performers

**You can use these queries as templates** for your own analysis or Power BI reports!

In [ ]:
print("\n" + "="*70)
print("📈 SAMPLE ANALYTICS\n")

print("💰 Daily Revenue (Last 10 Days):")
daily_revenue = fact_table.join(dim_date, "date_key") \
    .groupBy("date") \
    .agg(
        F.sum("revenue").alias("revenue"),
        F.sum("profit").alias("profit"),
        F.count("*").alias("transactions")
    ).orderBy(F.desc("date")).limit(10)
daily_revenue.show(truncate=False)

print("\n🏆 Top 10 Products by Revenue:")
top_products = fact_table.join(dim_product, "product_key") \
    .groupBy("product_name", "category") \
    .agg(
        F.sum("revenue").alias("revenue"),
        F.sum("quantity").alias("units")
    ).orderBy(F.desc("revenue")).limit(10)
top_products.show(truncate=False)

print("\n🏪 Top 10 Locations:")
location_perf = fact_table.join(dim_location, "location_key") \
    .groupBy("location_name", "city") \
    .agg(
        F.sum("revenue").alias("revenue"),
        F.count("*").alias("transactions")
    ).orderBy(F.desc("revenue")).limit(10)
location_perf.show(truncate=False)

print("\n" + "="*70)
print("✅ Generation Complete!")
print(f"\n🎉 Your {theme} {business_type} dataset is ready!")
print(f"   - {fact_row_count:,} transactions generated")
print(f"   - 4 tables written to lakehouse")
print(f"   - Ready for analytics, ML, and BI!")

## 🎓 Next Steps & Usage Ideas

### 🎯 What to do with your data:

**1. 📊 Build Power BI Dashboards**
- Open Power BI Desktop
- Connect to your Lakehouse (Direct Lake mode)
- Create stunning visualizations with your themed data!
- Your audience will love the creative names

**2. 💻 Run SQL Queries**

Try these sample queries in the SQL Analytics Endpoint:

```sql
-- Revenue by Category
SELECT 
    p.category,
    SUM(f.revenue) as total_revenue,
    COUNT(*) as transactions,
    AVG(f.revenue) as avg_transaction
FROM demo_fact_orders f  -- Change to your fact table name
JOIN demo_dim_product p ON f.product_key = p.product_key
GROUP BY p.category
ORDER BY total_revenue DESC;

-- Monthly Trends
SELECT 
    d.year,
    d.month_name,
    SUM(f.revenue) as revenue,
    SUM(f.profit) as profit
FROM demo_fact_orders f
JOIN demo_dim_date d ON f.date_key = d.date_key
GROUP BY d.year, d.month, d.month_name
ORDER BY d.year, d.month;

-- Top Performing Locations
SELECT 
    l.location_name,
    l.city,
    l.region,
    SUM(f.revenue) as total_revenue,
    COUNT(*) as transactions
FROM demo_fact_orders f
JOIN demo_dim_location l ON f.location_key = l.location_key
GROUP BY l.location_name, l.city, l.region
ORDER BY total_revenue DESC
LIMIT 20;
```

**3. 🤖 Train Machine Learning Models**
- Revenue forecasting with Prophet or ARIMA
- Customer segmentation clustering
- Product recommendation engines
- Anomaly detection on transaction patterns

**4. 🎨 Try Different Themes!**

Fun theme ideas:
- "Cyberpunk Street Food" (Restaurant)
- "Underwater Boutique" (Retail)
- "Time Travel Clinic" (Healthcare)
- "Dinosaur Theme Park" (Retail)
- "Steampunk Workshop" (Retail)
- "Alien Embassy" (Healthcare)

**5. 📈 Scale Up**
- Change `record_scale` to "large" for 500K transactions
- Test Power BI performance with larger datasets
- Benchmark your Spark queries

**6. 🎬 Create Demos & Presentations**
- The themed data makes presentations more engaging
- Audiences remember "Dragon's Breath Soup" better than "Product_042"
- Perfect for conference talks and workshops

---

## 🔧 Troubleshooting

**Problem: "Unterminated string" error**
- Solution: Increase `max_tokens` in Step 4 (try 8000 or 16000)

**Problem: Authentication failed (401)**
- Check your Azure OpenAI endpoint URL
- Verify your API key is correct
- Ensure your deployment name matches
- Remember to regenerate keys after demos!

**Problem: Table not found when refreshing**
- Use the dynamic refresh cell provided
- It auto-detects your fact table name

**Problem: Out of memory**
- Reduce `record_scale` to "small" or "medium"
- Increase your Fabric capacity if needed

---

## 📚 Resources

- [Microsoft Fabric Documentation](https://learn.microsoft.com/fabric/)
- [Power BI Direct Lake](https://learn.microsoft.com/power-bi/enterprise/directlake-overview)
- [Azure OpenAI Service](https://learn.microsoft.com/azure/ai-services/openai/)
- [PySpark Documentation](https://spark.apache.org/docs/latest/api/python/)

---

## 🎉 Share Your Creations!

Created something cool with this notebook? Share it!
- Post screenshots of your themed dashboards
- Share creative theme ideas
- Contribute improvements on GitHub

---

**Happy analyzing! 🚀**

*Remember: Always regenerate your Azure OpenAI keys after demonstrations!*